<a href="https://colab.research.google.com/github/Zoey234523/ML_assignment/blob/main/518_clean_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [8]:
df = pd.read_csv('train_data.csv')

In [4]:
df.columns

Index(['UserID', 'QuestionTiming', 'TimeUtc', 'CurrentGameMode', 'CurrentTask',
       'CurrentSessionLength', 'LastTaskCompleted', 'LevelProgressionAmount',
       'QuestionType', 'ResponseValue'],
      dtype='object')

**Step 1: Delete anomalies in the 'CurrentSessionLength' (<0, >7000)**

In [9]:
df = df[(df['CurrentSessionLength'] >= 0) & (df['CurrentSessionLength'] <= 7000)]


**Step 2:**
**Deal** **with CurrentGameMode's missing** **values**

Method: multinomial logical regression model (with UserID, ResponseValue)

*Result: The regression model below outputs the same result as imputing it with mode 'Career' :(*

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

user_id_encoder = LabelEncoder()
df['UserID_encoded'] = user_id_encoder.fit_transform(df['UserID'])

game_mode_encoder = LabelEncoder()
df['CurrentGameMode_encoded'] = game_mode_encoder.fit_transform(df['CurrentGameMode'].astype(str))

# split data and train logical-regression model
train_df = df[df['CurrentGameMode'].notnull()]
test_df = df[df['CurrentGameMode'].isnull()]

X_train = train_df[['UserID_encoded', 'ResponseValue']]
y_train = train_df['CurrentGameMode_encoded']

X_test = test_df[['UserID_encoded', 'ResponseValue']]

model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=200)
model.fit(X_train, y_train)

predicted = model.predict(X_test)
predicted_labels = game_mode_encoder.inverse_transform(predicted)

df.loc[df['CurrentGameMode'].isnull(), 'CurrentGameMode'] = predicted_labels

0


In [20]:
# check
print('Current number of missing values in CurrentGameMode:',df['CurrentGameMode'].isnull().sum())

category_counts = df['CurrentGameMode'].value_counts()
print("Number of each category in CurrentGameMode:")
print(category_counts)

Current number of missing values in CurrentGameMode: 0
Number of each category in CurrentGameMode:
CurrentGameMode
Career       127731
Special        4404
FreePlay       2265
Challenge        16
Name: count, dtype: int64


In [28]:
df.to_csv('impute_CGM.csv')

**Step 3: Impute missing values in CurrentTask**

Method:
1. sort by UserID and TimeUtc

2. For each row, if CurrentTask is missing:
  
*  Check both the previous and next non-null CurrentTask values. Compare the time differences to the current row.

*  Impute the missing value with the CurrentTask that has the shorter time difference.

3. If no appropriate previous or next task is found, we calculate the mode of the non-null CurrentTask values from the last week of data. These remaining missing values are then filled with this mode value. (the reason why I use mode of week's data is trying to keep the diversity of currenttask)

In [30]:
df = pd.read_csv('impute_CGM.csv')

In [31]:
from datetime import timedelta

df['TimeUtc'] = pd.to_datetime(df['TimeUtc'])

df.sort_values(by=['UserID', 'TimeUtc'], inplace=True)

start_date = df['TimeUtc'].min()
df['Week'] = ((df['TimeUtc'] - start_date).dt.days // 7)

# Function to fill missing CurrentTask values
def fill_missing(row):
    if pd.isnull(row['CurrentTask']):
        previous_data = df[(df['UserID'] == row['UserID']) & (~df['CurrentTask'].isnull()) & (df['TimeUtc'] < row['TimeUtc'])]
        next_data = df[(df['UserID'] == row['UserID']) & (~df['CurrentTask'].isnull()) & (df['TimeUtc'] > row['TimeUtc'])]

        previous_task = None
        next_task = None
        time_diff_prev = timedelta.max
        time_diff_next = timedelta.max

        if not previous_data.empty:
            time_diff_prev = row['TimeUtc'] - previous_data.iloc[-1]['TimeUtc']
            if time_diff_prev <= timedelta(hours=24):
                previous_level = previous_data.iloc[-1]['LevelProgressionAmount']
                if previous_level != 1:
                    previous_task = previous_data.iloc[-1]['CurrentTask']

        if not next_data.empty:
            time_diff_next = next_data.iloc[0]['TimeUtc'] - row['TimeUtc']
            if time_diff_next <= timedelta(hours=24):
                next_level = next_data.iloc[0]['LevelProgressionAmount']
                if next_level != 1:
                    next_task = next_data.iloc[0]['CurrentTask']

        if previous_task is not None and next_task is not None:
            if time_diff_prev <= time_diff_next:
                return previous_task
            else:
                return next_task
        elif previous_task is not None:
            return previous_task
        elif next_task is not None:
            return next_task

        # If neither previous_task nor next_task are suitable, use the mode of the current week
        week_data = df[(df['Week'] == row['Week']) & (~df['CurrentTask'].isnull())]
        if not week_data.empty:
            return week_data['CurrentTask'].mode()[0]

    return row['CurrentTask']

df['CurrentTask'] = df.apply(fill_missing, axis=1)

In [33]:
# check
print(df['CurrentTask'].isnull().sum())
print(df['CurrentGameMode'].isnull().sum())

0
0


In [34]:
df.to_csv('impute_CGM_CT.csv')

**Step 4: Impute LevelProgressionAmount's missing values**

Method:


*   Finds nearby non-missing records of the same user's same task within a 24-hour window. If such nearby records exist, it performs linear interpolation to estimate the missing value based on the time. If no nearby records exist, it fills the missing value with the median 'LevelProgressionAmount' of the user.
*   After applying first method, there are still 249 missing values (those currenttask are all 'RECREATIONGROUND_PLAYGROUND'). So for those whose currentsessionlength < 5, use this category's mdeian, >5 another median

In [35]:
df['TimeUtc'] = pd.to_datetime(df['TimeUtc'])
df.sort_values(by=['UserID', 'CurrentTask', 'TimeUtc'], inplace=True)

def fill_missing_values(df):
    filled_amount = []
    for i in range(len(df)):
        if pd.isnull(df.iloc[i]['LevelProgressionAmount']):
            # Find nearby non-missing records of the same user, task, and within a 24-hour window
            near_rows = df[(df['UserID'] == df.iloc[i]['UserID']) &
                           (df['CurrentTask'] == df.iloc[i]['CurrentTask']) &
                           (df['TimeUtc'] >= df.iloc[i]['TimeUtc'] - pd.Timedelta(hours=24)) &
                           (df['TimeUtc'] <= df.iloc[i]['TimeUtc'] + pd.Timedelta(hours=24)) &
                           (~df['LevelProgressionAmount'].isnull())]
            if not near_rows.empty:
                # Use linear interpolation to fill the missing value
                x = near_rows['TimeUtc'].values.astype(np.int64)
                y = near_rows['LevelProgressionAmount'].values
                x_interp = df.iloc[i]['TimeUtc'].value
                filled_value = np.interp(x_interp, x, y, left=0, right=1)
                filled_amount.append(max(min(filled_value, 1), 0))  # Limit between 0 and 1
            else:
                # Fill the missing value with the median LevelProgressionAmount of the user
                median_value = df[(df['UserID'] == df.iloc[i]['UserID']) &
                                  (~df['LevelProgressionAmount'].isnull())]['LevelProgressionAmount'].median()
                filled_amount.append(median_value)
        else:
            filled_amount.append(df.iloc[i]['LevelProgressionAmount'])
    return filled_amount

df['FilledLevelProgressionAmount'] = fill_missing_values(df)

# Calculate LevelProgressionAmount's mean
filtered1_df = df[(df['CurrentTask'] == 'RECREATIONGROUND_PLAYGROUND') & (df['CurrentSessionLength'] < 5)]
mean_value_1 = filtered1_df['LevelProgressionAmount'].mean()
filtered2_df = df[(df['CurrentTask'] == 'RECREATIONGROUND_PLAYGROUND') &
                 (df['CurrentSessionLength'] <= 20) &
                 (df['CurrentSessionLength'] >= 5)]
mean_value_2 = filtered2_df['LevelProgressionAmount'].mean()

df.loc[df['FilledLevelProgressionAmount'].isnull() & (df['CurrentSessionLength'] < 5), 'FilledLevelProgressionAmount'] = mean_value_1
df.loc[df['FilledLevelProgressionAmount'].isnull() & (df['CurrentSessionLength'] >= 5), 'FilledLevelProgressionAmount'] = mean_value_2


0


In [ ]:
# check
print(df['FilledLevelProgressionAmount'].isnull().sum())

In [36]:
df.to_csv('impute_CGM_CT_LPA.csv')

**step 5: impute missing values of LastTaskCompleted**

Method:


1.  For the same UserID's same CurrentTask, we assume that LastTaskCompleted is likely to be the same. If there are over 2 continus missing values(just left it as null)
2.  After applying method1, the remaining missing values can't infer from UserID and CureentTask due to lack of these data. So we further use Randomforest (add CurrentGameMode and CurrentTask) to predict it



In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

df['TimeUtc'] = pd.to_datetime(df['TimeUtc'])
df.sort_values(by=['UserID', 'TimeUtc'], inplace=True)

def fill_last_task(df):

    filled_last_task = []


    for i in range(len(df)):
        user_id = df.iloc[i]['UserID']
        current_time = df.iloc[i]['TimeUtc']
        current_task = df.iloc[i]['CurrentTask']

        # check if missing is continus
        if pd.isnull(df.iloc[i]['LastTaskCompleted']):
            if i > 1 and pd.isnull(df.iloc[i-1]['LastTaskCompleted']) and pd.isnull(df.iloc[i-2]['LastTaskCompleted']):
                filled_last_task.append(np.nan)
                continue

            previous_data = df[(df['UserID'] == user_id) & (~df['LastTaskCompleted'].isnull()) & (df['TimeUtc'] < current_time)]
            next_data = df[(df['UserID'] == user_id) & (~df['LastTaskCompleted'].isnull()) & (df['TimeUtc'] > current_time)]

            fill_value = np.nan

            if not previous_data.empty:
                prev_time = previous_data.iloc[-1]['TimeUtc']
                prev_task = previous_data.iloc[-1]['LastTaskCompleted']
                if current_time - prev_time <= pd.Timedelta(hours=24) or df.iloc[i]['CurrentTask'] == previous_data.iloc[-1]['CurrentTask']:
                    fill_value = prev_task

            if not next_data.empty:
                next_time = next_data.iloc[0]['TimeUtc']
                next_task = next_data.iloc[0]['LastTaskCompleted']
                if next_time - current_time <= pd.Timedelta(hours=24) or df.iloc[i]['CurrentTask'] == next_data.iloc[0]['CurrentTask']:
                    fill_value = next_task

            filled_last_task.append(fill_value)
        else:
            filled_last_task.append(df.iloc[i]['LastTaskCompleted'])

    return filled_last_task

df['FilledLastTaskCompleted'] = fill_last_task(df)

# Use Randomforest algorithm
features = ['CurrentTask', 'CurrentGameMode', 'LastTaskCompleted']
target = 'FilledLastTaskCompleted'

label_encoders = {}
for feature in features:
    if df[feature].dtype == 'object':
        le = LabelEncoder()
        df[feature] = le.fit_transform(df[feature].astype(str))
        label_encoders[feature] = le

train_data = df.dropna(subset=features + [target])
X = train_data[features]
y = train_data[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')

missing_data = df[df[target].isnull()]
missing_features = missing_data[features]

predicted_values = model.predict(missing_features)

df.loc[df[target].isnull(), target] = predicted_values



Accuracy: 0.8137567745530061
        Unnamed: 0 UserID    QuestionTiming             TimeUtc  \
0                0     p1    User Initiated 2022-08-18 22:55:27   
1                1     p1  System Initiated 2022-08-18 23:38:31   
2                2     p1    User Initiated 2022-08-18 23:39:24   
3                3     p1  System Initiated 2022-08-18 23:45:01   
4                4     p1  System Initiated 2022-08-18 23:51:22   
...            ...    ...               ...                 ...   
134411       70401  p9999  System Initiated 2022-10-12 03:44:44   
134412       70402  p9999  System Initiated 2022-10-12 04:44:06   
134413       70403  p9999  System Initiated 2022-10-12 05:01:29   
134414       70404  p9999  System Initiated 2022-10-12 05:09:22   
134415       70405  p9999  System Initiated 2022-10-12 05:24:06   

        CurrentGameMode  CurrentTask  CurrentSessionLength  LastTaskCompleted  \
0                     0           31                     2                 51   
1   

In [ ]:
df.to_csv('final_data.csv')

In [81]:
df1 = pd.read_csv('impute_CGM_CT_LPA.csv')
df2 = pd.read_csv('final_data.csv')

In [82]:
# replace df2's currentgamemode and currenttask with df1's
df1.sort_values(by=['UserID', 'TimeUtc'], inplace=True)
df1.reset_index(drop=True, inplace=True)
df1.index.name = 'NewIndexName'
print(df1)



              Unnamed: 0.1  Unnamed: 0 UserID    QuestionTiming  \
NewIndexName                                                      
0                        0           0     p1    User Initiated   
1                        1           1     p1  System Initiated   
2                        2           2     p1    User Initiated   
3                        3           3     p1  System Initiated   
4                        4           4     p1  System Initiated   
...                    ...         ...    ...               ...   
134411              134411       70401  p9999  System Initiated   
134412              134412       70402  p9999  System Initiated   
134413              134413       70403  p9999  System Initiated   
134414              134414       70404  p9999  System Initiated   
134415              134415       70405  p9999  System Initiated   

                          TimeUtc CurrentGameMode  \
NewIndexName                                        
0             2022-08-

In [83]:
df2.sort_values(by=['UserID', 'TimeUtc'], inplace=True)
df2.reset_index(drop=True, inplace=True)
df2.index.name = 'NewIndexName'
print(df2)

              Unnamed: 0.1  Unnamed: 0 UserID    QuestionTiming  \
NewIndexName                                                      
0                        0           0     p1    User Initiated   
1                        1           1     p1  System Initiated   
2                        2           2     p1    User Initiated   
3                        3           3     p1  System Initiated   
4                        4           4     p1  System Initiated   
...                    ...         ...    ...               ...   
134411              134411       70401  p9999  System Initiated   
134412              134412       70402  p9999  System Initiated   
134413              134413       70403  p9999  System Initiated   
134414              134414       70404  p9999  System Initiated   
134415              134415       70405  p9999  System Initiated   

                          TimeUtc  CurrentGameMode  CurrentTask  \
NewIndexName                                                 

In [84]:
df2['CurrentGameMode'] = df1['CurrentGameMode']
df2['CurrentTask'] = df1['CurrentTask']
df2['LastTaskCompleted'] = df2['FilledLastTaskCompleted']

print(df2)

              Unnamed: 0.1  Unnamed: 0 UserID    QuestionTiming  \
NewIndexName                                                      
0                        0           0     p1    User Initiated   
1                        1           1     p1  System Initiated   
2                        2           2     p1    User Initiated   
3                        3           3     p1  System Initiated   
4                        4           4     p1  System Initiated   
...                    ...         ...    ...               ...   
134411              134411       70401  p9999  System Initiated   
134412              134412       70402  p9999  System Initiated   
134413              134413       70403  p9999  System Initiated   
134414              134414       70404  p9999  System Initiated   
134415              134415       70405  p9999  System Initiated   

                          TimeUtc CurrentGameMode  CurrentTask  \
NewIndexName                                                  

In [90]:
df2.to_csv('final_clean_data.csv')